# ETC Calibration

In [46]:
import numpy as np
from os import listdir
import sewpy
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy import units as u
import math
import scipy.integrate as integ
import matplotlib.pyplot as plt
from astropy.io import fits

Specify camera and target name and test frame location:

In [47]:
camera_name = 'PL09000'
target_name = 'SA 92-508'

frame_directory = './SA92508-all/'
frames = listdir(frame_directory) # load frames

Define function to match target temperature to nearest template spectrum:

In [48]:
def find_nearest(arr, val):
    """Function to find nearest value in an array to a given value
    :param arr: array of values to be searched
    :param val: value to be matched
    :return: index of item in array that is closest to the required value
    """
    arr = np.array(arr) # convert to numpy array
    idx = (abs(arr - val)).argmin() # find index of closest value
    return idx

Define function to create equation of a 2D Gaussian:

In [49]:
def twodGauss(x, y, A, sigma):
    """Defines the function of a 2d Gaussian function that encloses the total flux
    :param x: x coordinates to be integrated over
    :param y: y coordinates to be intagrated over
    :param A: amplitude of the Gaussian curve
    :param sigma: width of the Gaussian, specified by the site seeing
    :return: function of 2d Gaussian
    """
    return A*np.exp(-1/(2*sigma*sigma)*(x*x+y*y))

Initialise telescope characterisitics:

In [50]:
telescope_eff = 0.96*0.96*(1-0.005) # efficiency of the mirrors
telescope_r = 0.305 # radius; m
telescope_area = np.pi*telescope_r*telescope_r # telescope area; m^2
telescope_obs = 0.47 # telescope obscuration
telescope_total = telescope_eff*telescope_area*(1-telescope_obs) # total effct of telescope; m^2

Load spectral type/temp data from lookup table, specify standard filter data:

In [51]:
table_types = np.genfromtxt('colour-temp.txt',unpack=True,usecols=(0),dtype='str') #load spectral types
table_temps = np.genfromtxt('colour-temp.txt',unpack=True,usecols=(1)) #load corresponding temperatures

B_zp = -20.45 #known zero point for B

#standard filters, centres, widths, and zero points
filters = ['U','B','V','R','I','J','H','K']
centres = [3650,4450,5510,6580,8060,12200,16300,21900]
widths = [660,940,880,1380,1490,2130,3070,3900]
zeros = [-20.94,-20.45,-21.12,-21.61,-22.27,-23.80,-24.80,-26.00]

Load temperatures for template spectra:

In [52]:
spectrum_directory = './spectra/'
spectra = listdir(spectrum_directory)

#extract temperatures from filenames and create array
temps = []
for spectrum in spectra:
    temp = int(spectrum.split('-')[0][3:])
    temps.append(temp)
temps = np.sort(temps) # sort in ascending order

Load data for filters to be calibrated, set camera and site characteristics:

In [53]:
# load filter data in \AA
fcentres, fwidths = np.genfromtxt('filters.dat', unpack=True, usecols=(1,2))
fnames = np.genfromtxt('filters.dat', unpack=True, usecols=(0), dtype='str').tolist()

pixel_width_um = 12 #pixel width; um
plate_scale = 0.621 #plate scale; as/pixel
pixel_width_as = pixel_width_um*plate_scale #pixel width; as

site_seeing = 3 #seeing at site; as

Define SExtractor object to obtain RA/Dec and perform photometry:

In [54]:
sew = sewpy.SEW(params=['X_IMAGE','Y_IMAGE','FWHM_WORLD','ALPHA_J2000','DELTA_J2000','FLUX_BEST'],
                config={'DETECT_MINAREA':20, 'PHOT_APERTURES':'5,10,20,50'},
                sexpath='sex')

Create Simbad object to obtain target coordinates, spectral type and B magnitude:

In [55]:
custom_sim = Simbad()
custom_sim.add_votable_fields('flux(B)','sp','coordinates') # add required fields

simdata = custom_sim.query_object(target_name) # run query


raw_ra, raw_dec = simdata['RA'][0], simdata['DEC'][0] #extract coordinates
target_coords = SkyCoord(raw_ra, raw_dec, unit=(u.hourangle, u.deg)) # create SkyCoord object
# extract coords in degrees
ra = target_coords.ra.deg
dec = target_coords.dec.deg

Bmag = simdata[0]['FLUX_B'] # extract B magnitude

Extract target spectral type, convert to temperature, determine and then load corresponding template spectrum:

In [56]:
sp = str(simdata[0]['SP_TYPE'])[2:4]#obtain 2-character spectral type from data table
temp_val = table_temps[np.where(table_types == sp+'V')[0][0]]  #convert spectral type to tempterature using lookup table
#loopup table only has V spectral classes, so add V as approximation

t_index = find_nearest(temps, temp_val) # find nearest available temp to value
# find corresponding file 
for spectrum in spectra:
    # check filenames for nearest temp
    if np.int(spectrum.split('-')[0][3:]) == np.int(temps[t_index]):
        spectrum_file = spectrum

# load file        
wave, SED = np.genfromtxt(spectrum_directory+spectrum_file, unpack=True, usecols=(0,1))
wave *= 1e4 # convert to \AA
wave *= 1e-10 # convert to m
SED *= 10/1e-6 # convert to W/m^2/m

Calculate multiplicative offset $\alpha$ between calculated and actual zero point for each of the standard filters, then define a polynomial relationship the describes the relationship between filter effective wavelength and $\alpha$:

### Todo: Improve to use ascii files for standard filters instead of step functions?

In [57]:
alphas = [] #initialise array
order = 3 #set polynomial order

for i in range(0,len(filters)): #loop across filters
    loop_band = np.zeros(len(wave)) # new band array
    # find edges in wavelength space
    loop_band_min = (centres[i] - widths[i]/2)*1e-10 # m
    loop_band_max = (centres[i] + widths[i]/2)*1e-10 # m
    # find corresponding indexes
    loop_start_idx = abs(wave - loop_band_min).argmin()
    loop_end_idx = abs(wave - loop_band_max).argmin()
    #create step function in waveband range
    for j in range(loop_start_idx, loop_end_idx):
        loop_band[j] = 1.0
        
    loop_trans = SED*loop_band # isolate transmission in waveband
    loop_flux = integ.simps(loop_trans, wave, dx=10e-10) # find total
    
    z_calc = 2.5*math.log10(loop_flux) # calculate zero point
    new_alpha = zeros[i]/z_calc # find proportionality between calculated and known zero points
    alphas.append(new_alpha) # add to array
# fit polynomial to alpha values
alpha_poly = np.poly1d(np.polyfit(centres, alphas, order))

Find flux in B band from template spectrum:

In [58]:
Bmid = 4450
Bwidth = 940
Bband = np.zeros(len(wave)) #new, bespoke band array
#find edges of band
Bband_min = (Bmid - Bwidth/2)*1e-10
Bband_max = (Bmid + Bwidth/2)*1e-10
#find corresponding indexes
Bstart_idx = abs(wave - Bband_min).argmin()
Bend_idx = abs(wave - Bband_max).argmin()
#create stpe function in band
for i in range(Bstart_idx, Bend_idx):
    Bband[i] = 1.0
    
Btrans = SED*Bband # isolate flux in B band
Bflux = integ.simps(Btrans, wave, dx=10e-10) # find total

Load CCD efficiency polynomial from file and initialise across wavelength range:

In [59]:
# load polynomial coefficients from file
ccd_poly_coeffs = np.genfromtxt('./CCD-effs/'+camera_name+'.poly', usecols=(1))
ccd_poly = np.poly1d(ccd_poly_coeffs) # create polynonmial object
ccd_response_curve = ccd_poly(wave) # initialise across wavelength range

Calculate theoretical fluxes in each waveband being calibrated using flux in B band and filter zero points. Then for each waveband account for telescope geometry and distribution of flux over a single pixel to obtain a final theoretical flux being recieved by a single pixel.

Then, for each test frame in each waveband, carry out aperture photometry using SExtractor and extract a flux rate being received from the target using the exposure time of each frame and CCD characteristics:

In [60]:
for i in range(0,len(fcentres)): # loop across filters
    coefficients = [] # array for coefficients
    # find filter centre and width
    mid = fcentres[i]
    wid = fwidths[i]
    band = np.zeros(len(wave)) # new band of zeros
    #find edges of waveband
    band_min = (mid - wid/2)*1e-10
    band_max = (mid + wid/2)*1e-10
    #find corresponding indexes in wavelength array
    start_idx = abs(wave - band_min).argmin()
    end_idx = abs(wave - band_max).argmin()
    #create step function within waveband
    for j in range(start_idx, end_idx):
        band[j] = 1.0
        
    alpha = alpha_poly(mid) # obtain alpha value from polynomial

    transmission = SED*band # isolate transmission within band
    flux = integ.simps(transmission, wave, dx=10e-10) # find total
    z_calc = 2.5*math.log10(flux) # calculate zero point
    zp = alpha*z_calc # correct using alpha
    # calculate magnitude in current filter
    Fmag = Bmag + 2.5*math.log10(Bflux/flux) - B_zp + zp
    Fflux = 10**((zp - Fmag)/2.5) # find corresponding flux; W/m^2/m
    flux = Fflux*telescope_total # adjust for telescope effects W/m
    
    #trim ccd response curve to waveband
    ccd_response_band = ccd_response_curve[start_idx:end_idx]
    ccd_response = np.mean(ccd_response_band) # take average response within band
    flux *= ccd_response # calculate detected flux
    
    gauss_amp = flux/(2*np.pi*site_seeing*site_seeing) #  determine amplitude of gaussian
    # integrate gaussian specified by amplitude and site seeing over a single pixel
    theory_flux = integ.dblquad(twodGauss,-pixel_width_as/2,pixel_width_as/2,-pixel_width_as/2,pixel_width_as/2, args=(gauss_amp,site_seeing))[0] # J/s (W)
    
    for frame in frames: # loop across all test frames
        fits_header = fits.open(frame_directory+frame)[0].header #open fits header
        filt = fits_header['FILTER'] # obtain filter used
        if filt == fnames[i]: # only proceed if observed filter matches current filter
            sewdata = sew(frame_directory+frame) # run SExtractor
            sewtable = sewdata['table'] # obtain data table
            # obtain desired columns from table
            flux_column = sewtable['FLUX_BEST']
            ra_column = sewtable['ALPHA_J2000']
            dec_column = sewtable['DELTA_J2000']
            # find target index by matching coordinates
            target_idx = np.where(np.logical_and(abs(ra_column - ra)< 0.01, abs(dec_column - dec) < 0.01))[0]

            exptime = fits_header['EXPTIME'] # obtain exposure time of image

            filter_idx = fnames.index(filt) # find index of filter in filter array
            # load filter data
            mid = fcentres[filter_idx]
            wid = fwidths[filter_idx]
            #define wavelength array in waveband
            fband = np.arange((mid-wid/2)*1e-10, (mid+wid/2)*1e-10, 10e-10)

            ccd_curve = ccd_poly(fband) # initialise ccd polynomial in waveband
            ccd_avgres = np.mean(ccd_curve) # take average response in band; photons/e-
            
            # calculate average energy of photon in band based on central wavelength
            h = 6.63e-34 # Planck constant; m^2kg/s
            c = 3e8 # speed of light; ms^-1
            avgE = h*c/(mid*1e-10) # energy of single photon; J

            counts = flux_column[target_idx][0] # total ADU from target
            gain = 1.4 # e-/ADU
            electrons = counts*gain # e-
            photons = electrons/ccd_avgres # photons
            flux_tot = avgE*photons # J
            flux_rate = flux_tot/exptime # J/s (W)
            
            new_coeff = theory_flux/flux_rate
            coefficients.append(new_coeff)
            
    mean_coeff = np.mean(coefficients)
    #print(coefficients)
    print('Filter: '+fnames[i]+' Coeff: '+str(mean_coeff))

Ouch, SExtractor complains :
b''


Filter: r' Coeff: 0.6940282877047105
Filter: g' Coeff: 0.6454135579945701
Filter: i' Coeff: 0.5213640811275398
